In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
from sklearn.metrics import classification_report, confusion_matrix
#import seaborn as sns
import cv2

import glob


# example of zoom image augmentation
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

In [2]:
main_path = R"D:/chest_xray"

In [3]:
train_path = os.path.join(main_path,"train")
test_path=os.path.join(main_path,"test")
val_path=os.path.join(main_path,"val")

In [4]:
pneumonia_train_images = glob.glob(train_path+"/PNEUMONIA/*.jpeg")
normal_train_images = glob.glob(train_path+"/NORMAL/*.jpeg")

In [5]:
pneumonia_test_images = glob.glob(test_path+"/PNEUMONIA/*.jpeg")
normal_test_images = glob.glob(test_path+"/NORMAL/*.jpeg")

In [6]:
pneumonia_val_images = glob.glob(val_path+"/PNEUMONIA/*.jpeg")
normal_val_images = glob.glob(val_path+"/NORMAL/*.jpeg")

In [7]:
data = pd.DataFrame(np.concatenate([[0]*len(normal_train_images) , [1] *  len(pneumonia_train_images)]),columns=["class"])

In [8]:
#sns.countplot(data['class'],data=data)

In [9]:
Normal_train=[]
pneumonia_trian =[]

In [10]:
import cv2
from tqdm import tqdm
for dirs in tqdm(normal_train_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    Normal_train.append(np.array(resized))

100%|██████████| 1341/1341 [00:21<00:00, 62.83it/s]


In [11]:
import cv2
from tqdm import tqdm
for dirs in tqdm(pneumonia_train_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    pneumonia_trian.append(np.array(resized))

100%|██████████| 3875/3875 [00:21<00:00, 176.56it/s]


In [12]:
Normal_train_array=[]

In [13]:
for i in tqdm(range(0,len(Normal_train))):
    #print(a.shape)
    #print(Normal_train[i].shape)
    Normal_train_array.append(Normal_train[i])

100%|██████████| 1341/1341 [00:00<?, ?it/s]


In [14]:
ind = 0
from tqdm import tqdm
for dirs in tqdm(normal_train_images):
    ind=ind+1
    img = cv2.imread(dirs) 
    data = img_to_array(img)
    samples = expand_dims(data, 0)
    
    if ind%2 ==0:
        continue
    else:
        datagen = ImageDataGenerator(zoom_range=[0.8,1.0])
        it = datagen.flow(samples, batch_size=1)
        for i in range(2):
            batch = it.next()
            image = batch[0].astype('uint8')
            resized = cv2.resize(image, (200,200))
            img2gray = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY)
            Normal_train_array.append(img2gray)

100%|██████████| 1341/1341 [06:23<00:00,  3.50it/s]


In [15]:
Normal_train_array_X = np.array(Normal_train_array)

In [16]:
Normal_train_array_y = [0]*len(Normal_train_array_X)

In [17]:
len(Normal_train_array_y)

2683

In [18]:
pneumonia_trian_array_X = np.array(pneumonia_trian)

In [19]:
pneumonia_train_array_y = [1]*len(pneumonia_trian_array_X)

In [20]:
X_train = np.concatenate((Normal_train_array_X, pneumonia_trian_array_X), axis=0)

In [21]:
y_train = np.concatenate((np.array(Normal_train_array_y), np.array(pneumonia_train_array_y)), axis=0)

In [22]:
y_train.shape

(6558,)

In [23]:
X_train.shape

(6558, 200, 200)

## Read testing Image

In [24]:
Normal_test=[]
pneumonia_test =[]
Normal_test_array=[]

In [25]:
import cv2
from tqdm import tqdm
for dirs in tqdm(normal_test_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    Normal_test.append(np.array(resized))

100%|██████████| 234/234 [00:03<00:00, 75.21it/s] 


In [26]:
import cv2
from tqdm import tqdm
for dirs in tqdm(pneumonia_test_images):
    img = cv2.imread(dirs)    
    img2gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(img2gray, (200,200))
    #print(resized.shape)
    pneumonia_test.append(np.array(resized))

100%|██████████| 390/390 [00:01<00:00, 198.60it/s]


In [27]:
for i in tqdm(range(0,len(Normal_test))):
    #print(a.shape)
    #print(Normal_train[i].shape)
    Normal_test_array.append(Normal_test[i])

100%|██████████| 234/234 [00:00<?, ?it/s]


In [28]:
Normal_test_array_X = np.array(Normal_test_array)

In [29]:
Normal_test_array_y = [0]*len(Normal_test_array_X)

In [30]:
pneumonia_test_array_X = np.array(pneumonia_test)

In [31]:
pneumonia_test_array_y = [1]*len(pneumonia_test_array_X)

In [32]:
X_test = np.concatenate((Normal_test_array_X, pneumonia_test_array_X), axis=0)

In [33]:
y_test = np.concatenate((np.array(Normal_test_array_y), np.array(pneumonia_test_array_y)), axis=0)

In [34]:
y_test.shape

(624,)

In [35]:
X_test.shape

(624, 200, 200)

In [36]:
from tensorflow.keras.utils import to_categorical
# one hot encode output variable
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [37]:
y_train = np.argmax(y_train, axis = 1)
y_test = np.argmax(y_test, axis = 1)

In [38]:
# stacked generalization with neural net meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from numpy import argmax

# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'models/model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense(32, activation='relu')(merge)
    hidden = Dense(16, activation='relu')(hidden)
    output = Dense(2, activation='softmax')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)
    # plot graph of ensemble
    #plot_model(model, show_shapes=True, to_file='model_graph.png')
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
    print("=================== Working on Meta classifier ===================")
    # prepare input data
    #filename = 'models/final.h5'
    X = [inputX for _ in range(len(model.input))]
    # encode output data
    inputy_enc = inputy #to_categorical(inputy)
    print(inputy_enc.shape)
    # fit model
    model.fit(X, inputy_enc, epochs=100, verbose=1, batch_size=150, validation_split=0.2)
    #model = load_model(filename)
    
    
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]
    # make prediction
    return model.predict(X, verbose=0)

In [39]:
from tensorflow.keras.callbacks import LearningRateScheduler

# This is a sample of a scheduler I used in the past
def lr_scheduler(epoch, lr):
    if epoch % 100 == 0 and epoch:
        lr = lr - lr*0.1
        return lr
    return lr

## Start fitting

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras as keras
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense, Conv2D, MaxPooling2D, MaxPool2D


def Model_fitting(X, y, index, epoch):
    # define the keras model
    X = X.reshape(X.shape[0],X.shape[1], X.shape[2], 1)
    
    model = ResNet50V2(include_top=False, weights=None, input_tensor=Input(shape=((X.shape[1], X.shape[2], 1))))
    
    x = model.output
    x = Flatten()(x)
    x = Dense(64, activation = 'sigmoid')(x)
    x = Dense(2, activation = "sigmoid")(x)
    model = tensorflow.keras.Model(model.input, x)
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.AUC()])
    model.fit(X, y, epochs=epoch, batch_size=250, 
              verbose = 1, 
              validation_split=0.1,
              callbacks = [LearningRateScheduler(lr_scheduler, verbose=0),
                                    tensorflow.keras.callbacks.ModelCheckpoint(filename,
                                                                               monitor='loss',
                                                                               save_best_only=True,
                                                                               mode='min',
                                                                               verbose=0)])
    model = load_model(filename)
    return model.predict(X)

### FItting stacked models

In [41]:
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'models/model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

In [42]:
from tensorflow.keras.utils import to_categorical

In [43]:
from sklearn.metrics import confusion_matrix
    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel()

    
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    
    
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

In [44]:
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import warnings
import timeit
import tracemalloc
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from pathlib import Path
from tensorflow.keras import layers
import tensorflow
from tensorflow.keras.models import load_model
warnings.filterwarnings('ignore')
# split 10 fold of undersampling
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve


max_auc = 0.93

for time in range(0, 10):
    rsFile = "Result_OverIteration_Bagging_Numeric/NumberOfIteration_"+str(time)+".csv"
    print("working on num_Interation: " + str(time))
    f2=open(rsFile,"w")
    f2.write('Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    for num_Interation in ([8]):
        for epoch in ([50]):
            
            # ============== Starting training
            start = timeit.default_timer()
            tracemalloc.start()

            ## Start training
            for i in range(0, num_Interation):
                # save model
                filename = 'models/model_' + str(i + 1) + '.h5'
                # fit model
                model = Model_fitting(X_train, to_categorical(y_train), filename, epoch)
                #model = fit_model(trainX, trainy)
                #model.save(filename)
                print('>Saved %s' % filename)
            current, peak = tracemalloc.get_traced_memory()
            stop = timeit.default_timer()
            Time_train=stop - start
            Train_Peak_RAM = peak / 10**6
            #print('traning time: ', Time_train)
            #print('Peak_Time:', Train_Peak_RAM)


            # ============== Starting predicting
            members = load_all_models(num_Interation)
            # define ensemble model
            stacked_model = define_stacked_model(members)
            #stacked_model.name = 'stacked_model_iter_'+str(num_Interation)+'_Epoc_'+str(epoch)
            # fit stacked model on test dataset
            fit_stacked_model(stacked_model, X_train, to_categorical(y_train))


            start = timeit.default_timer()
            tracemalloc.start()

            predicted = predict_stacked_model(stacked_model, X_test)

            stop = timeit.default_timer()
            Time_predict=stop - start
            current, peak = tracemalloc.get_traced_memory()
            Peak_Time_prediction = peak / 10**6


            auc=roc_auc_score(y_test, predicted[:, 1])
            print('y_test.shape ' ,y_test.shape)
            print('==========================================')


            if max_auc <= auc:
                #max_auc = auc
                print('===================: ')
                print('======= AUC ==============: ', auc)
                print('=====AUC===AUC=====: ')
                print('===AUC========AUC==: ')
                print('=AUC===========AUC=: ')
                print('=AUC===========AUC=: ')
                print('=AUC===========AUC=: ')
                print('===================: ')
                print('===================: ')



            thres = predicted[:, 1].min()
            while (thres <= predicted[:, 1].max()):
                thres = thres+(predicted[:, 1].max()/500)
                pred = []
                for i in range(len(predicted)):
                    if predicted[i][1]>thres:
                        pred.append(1)
                    else:
                        pred.append(0)

                pred = np.array(pred)      

                tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(y_test, pred)
                #auc = roc_plot(y_test, yhat)
                f2.write(str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")

            del(stacked_model)
    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       
        

working on num_Interation: 0
Epoch 1/50
24/24 [==============================] - 27s 603ms/step - loss: 0.7858 - auc: 0.8585 - val_loss: 1.5173 - val_auc: 0.0000e+00
Epoch 2/50
24/24 [==============================] - 10s 421ms/step - loss: 0.4042 - auc: 0.9600 - val_loss: 2.0149 - val_auc: 0.0000e+00
Epoch 3/50
24/24 [==============================] - 10s 421ms/step - loss: 0.2874 - auc: 0.9757 - val_loss: 1.7490 - val_auc: 0.0387
Epoch 4/50
24/24 [==============================] - 10s 421ms/step - loss: 0.2195 - auc: 0.9834 - val_loss: 0.8855 - val_auc: 0.5437
Epoch 5/50
24/24 [==============================] - 10s 422ms/step - loss: 0.1833 - auc: 0.9881 - val_loss: 0.3806 - val_auc: 0.9134
Epoch 6/50
24/24 [==============================] - 10s 422ms/step - loss: 0.1652 - auc: 0.9912 - val_loss: 0.0849 - val_auc: 1.0000
Epoch 7/50
24/24 [==============================] - 10s 423ms/step - loss: 0.1434 - auc: 0.9934 - val_loss: 1.1639 - val_auc: 0.5052
Epoch 8/50
24/24 [==============

In [45]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1], X_train.shape[2], 1)

In [46]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [47]:
X_train.shape

(6558, 200, 200, 1)

In [48]:
y_train.shape

(6558, 2)

In [49]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1], X_test.shape[2], 1)

In [50]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1], X_test.shape[2], 1)

In [51]:
from sklearn.metrics import confusion_matrix    
import math
def conf_matrix(y_real, y_predict):
    tn, fp, fn, tp = confusion_matrix(y_real, y_predict).ravel() 
    
    acc= (tp + tn)/(tp+tn+fp+fn)
    pre= tp/(tp+fp)
    Recall=  (tp)/(tp+fn)
    spec= (tn)/(tn+fp)
    F1score= (2*(tp)/(2*tp+fp+fn))
    mcc= ((tp*tn)-(fp*fn))/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    
    
    return tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc

In [52]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [53]:
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.utils import to_categorical
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
from tqdm import tqdm
import warnings
import timeit
import tracemalloc
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from pathlib import Path
from tensorflow.keras import layers
    
warnings.filterwarnings('ignore')
# split 10 fold of undersampling
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve


max_auc = 0.93
for num_Interation in range(2, 50):
    rsFile = "/content/drive/MyDrive/Colab Notebooks/Ensemble_Journal/Result_OverIteration_Bagging_Numeric/Image/NumberOfBag_"+str(num_Interation)+".csv"
    '''if (Path(rsFile).is_file()):
        continue
    else:
        '''
    f2=open(rsFile,"w")
    f2.write('num_Interation, epoch, Time_train, Train_Peak_RAM, Time_predict, Peak_Time_prediction, thres, tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc, auc\n')
    for epoch in (10, 50, 100, 500, 1000):
        print("working on num_Interation: " + str(num_Interation) + "_with_epoch_"+str(epoch))
        
        
        
        # ============== Starting training
        start = timeit.default_timer()
        tracemalloc.start()

        ## Start training
        for i in range(0, num_Interation):
            # save model
            filename = 'models/model_' + str(i + 1) + '.h5'
            # fit model
            model = Model_fitting(X_train, y_train, filename, epoch)
            #model = fit_model(trainX, trainy)
            #model.save(filename)
            print('>Saved %s' % filename)
        current, peak = tracemalloc.get_traced_memory()
        stop = timeit.default_timer()
        Time_train=stop - start
        Train_Peak_RAM = peak / 10**6
        #print('traning time: ', Time_train)
        #print('Peak_Time:', Train_Peak_RAM)


        # ============== Starting predicting
        members = load_all_models(num_Interation)
        # define ensemble model
        stacked_model = define_stacked_model(members)
        #stacked_model.name = 'stacked_model_iter_'+str(num_Interation)+'_Epoc_'+str(epoch)
        # fit stacked model on test dataset
        fit_stacked_model(stacked_model, X_train, y_train)


        start = timeit.default_timer()
        tracemalloc.start()

        predicted = predict_stacked_model(stacked_model, X_test)

        stop = timeit.default_timer()
        Time_predict=stop - start
        current, peak = tracemalloc.get_traced_memory()
        Peak_Time_prediction = peak / 10**6


        auc=roc_auc_score(np.argmax(y_test, axis=1), predicted[:, 1])
        print('y_test.shape ' ,y_test.shape)
        print('==========================================')


        if max_auc <= auc:
            #max_auc = auc
            print('===================: ')
            print('======= AUC ==============: ', auc)
            print('=====AUC===AUC=====: ')
            print('===AUC========AUC==: ')
            print('=AUC===========AUC=: ')
            print('=AUC===========AUC=: ')
            print('=AUC===========AUC=: ')
            print('===================: ')
            print('===================: ')



        thres = predicted[:, 1].min()
        while (thres <= predicted[:, 1].max()):
            thres = thres+0.002
            pred = []
            for i in range(len(predicted)):
                if predicted[i][1]>thres:
                    pred.append(1)
                else:
                    pred.append(0)

            pred = np.array(pred)      

            tn, fp, fn, tp, acc, pre, Recall, spec, F1score, mcc = conf_matrix(np.argmax(y_test, axis=1), pred)
            #auc = roc_plot(y_test, yhat)
            f2.write(str(num_Interation)+", "+str(epoch)+", "+str(Time_train)+", "+str(Train_Peak_RAM)+", "+str(Time_predict)+", "+str(Peak_Time_prediction)+", "+str(thres)+", "+str(tn)+", "+str(fp)+", "+str(fn)+", "+str(tp)+", "+str(acc)+", "+str(pre)+", "+str(Recall)+", "+str(spec)+", "+str(F1score)+", "+str(mcc)+", "+str(auc)+"\n")

        del(stacked_model)
    f2.close()
    print('WRITING FILE SUCESSFULL ========!!!!!!!!!!!!!!!!!!!!!!')       
        

ModuleNotFoundError: No module named 'imblearn'